In [4]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import os
import shutil

In [5]:
patients_df = pd.read_csv(r"D:/Tarun/medical documents research paper/data 100/people.csv")
admissions_df = pd.read_csv(r"D:/Tarun/medical documents research paper/data 100/admissions.csv")
diagnoses_df = pd.read_csv(r"D:/Tarun/medical documents research paper/data 100/diagnosis.csv")
labs_df = pd.read_csv(r"D:/Tarun/medical documents research paper/data 100/lab_results.csv")

C:\Users\HP\AppData\Local\Temp/ipykernel_4580/1891431008.py:4: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  labs_df = pd.read_csv(r"D:/Tarun/medical documents research paper/data 100/lab_results.csv")


In [6]:
patients_df.head()

,PatientID,PatientGender,PatientDateOfBirth,PatientRace,PatientMaritalStatus,PatientLanguage,PatientPopulationPercentageBelowPoverty
0,FB2ABB23-C9D0-4D09-8464-49BF0B982F0F,Male,45:40.5,Unknown,Married,Icelandic,18.08
1,64182B95-EB72-4E2B-BE77-8050B71498CE,Male,51:12.9,African American,Separated,English,13.03
2,DB22A4D9-7E4D-485C-916A-9CD1386507FB,Female,04:20.7,Asian,Married,English,6.67
3,6E70D84D-C75F-477C-BC37-9177C3698C66,Male,45:29.6,White,Married,English,16.09
4,C8556CC0-32FC-4CA5-A8CD-9CCF38816167,Female,39:49.2,White,Married,English,18.20


In [7]:
admissions_df.head()    

,PatientID,AdmissionID,AdmissionStartDate,AdmissionEndDate
0,7A025E77-7832-4F53-B9A7-09A3F98AC17E,7,55:02.0,16:07.6
1,DCE5AEB8-6DB9-4106-8AE4-02CCC5C23741,1,57:04.0,22:29.7
2,DCE5AEB8-6DB9-4106-8AE4-02CCC5C23741,2,06:31.1,14:40.8
3,DCE5AEB8-6DB9-4106-8AE4-02CCC5C23741,3,42:45.3,57:42.1
4,886B5885-1EE2-49F3-98D5-A2F02EB8A9D4,1,20:46.1,24:56.0


In [8]:
labs_df.head()

,PatientID,AdmissionID,LabName,LabValue,LabUnits,LabDateTime,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,1A8791E3-A61C-455A-8DEE-763EB90C9B2C,1,URINALYSIS: RED BLOOD CELLS,1.8,rbc/hpf,36:17.9,NaN,NaN,NaN,NaN
1,1A8791E3-A61C-455A-8DEE-763EB90C9B2C,1,METABOLIC: GLUCOSE,103.3,mg/dL,35:52.4,NaN,NaN,NaN,NaN
2,1A8791E3-A61C-455A-8DEE-763EB90C9B2C,1,CBC: MCH,35.8,pg,50:11.8,NaN,NaN,NaN,NaN
3,1A8791E3-A61C-455A-8DEE-763EB90C9B2C,1,METABOLIC: CALCIUM,8.9,mg/dL,09:46.1,NaN,NaN,NaN,NaN
4,1A8791E3-A61C-455A-8DEE-763EB90C9B2C,1,CBC: RED BLOOD CELL COUNT,4.8,m/cumm,31:08.7,NaN,NaN,NaN,NaN


In [9]:
diagnoses_df.head()

,PatientID,AdmissionID,PrimaryDiagnosisCode,PrimaryDiagnosisDescription
0,80AC01B2-BD55-4BE0-A59A-4024104CF4E9,2,M01.X,Direct infection of joint in infectious and pa...
1,80AC01B2-BD55-4BE0-A59A-4024104CF4E9,3,D65,Disseminated intravascular coagulation [defibr...
2,80AC01B2-BD55-4BE0-A59A-4024104CF4E9,4,C92.1,"Chronic myeloid leukemia, BCR/ABL-positive"
3,80AC01B2-BD55-4BE0-A59A-4024104CF4E9,5,M05.51,Rheumatoid polyneuropathy with rheumatoid arth...
4,6A57AC0C-57F3-4C19-98A1-51135EFBC4FF,1,C91.00,Acute lymphoblastic leukemia not having achiev...


In [10]:
# Merge patients with admissions
patient_admissions_df = pd.merge(patients_df, admissions_df, on="PatientID")

# Merge the above with diagnoses
patient_admissions_diagnoses_df = pd.merge(
    patient_admissions_df, diagnoses_df, on=["PatientID", "AdmissionID"]
)

# Pivot the labs dataframe so that each lab type becomes a column
labs_pivot_df = labs_df.pivot_table(
    index=["PatientID", "AdmissionID"],
    columns="LabName",
    values=["LabValue", "LabUnits", "LabDateTime"],
)

# Flatten the multi-index columns
labs_pivot_df.columns = ["_".join(col).strip() for col in labs_pivot_df.columns.values]

# Merge the pivoted labs data with the rest of the data
final_df = pd.merge(
    patient_admissions_diagnoses_df,
    labs_pivot_df,
    on=["PatientID", "AdmissionID"],
    how="left",
)

In [14]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 367 entries, 0 to 366
Data columns (total 47 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   PatientID                                367 non-null    object 
 1   PatientGender                            367 non-null    object 
 2   PatientDateOfBirth                       367 non-null    object 
 3   PatientRace                              367 non-null    object 
 4   PatientMaritalStatus                     367 non-null    object 
 5   PatientLanguage                          367 non-null    object 
 6   PatientPopulationPercentageBelowPoverty  367 non-null    float64
 7   AdmissionID                              367 non-null    int64  
 8   AdmissionStartDate                       367 non-null    object 
 9   AdmissionEndDate                         367 non-null    object 
 10  PrimaryDiagnosisCode                     367 non-n

In [13]:
from datetime import datetime

# Calculate patient age at admission
final_df["PatientAge"] = final_df.apply(
    lambda row: (
        datetime.strptime(row["AdmissionStartDate"], "%Y-%m-%d")
        - datetime.strptime(row["PatientDateOfBirth"], "%Y-%m-%d")
    ).days
    // 365,
    axis=1,
)

# Optionally, you can create embeddings for the diagnosis descriptions if needed
# Here, we'll use a simple label encoding for demonstration (in practice, you might use word embeddings)
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
final_df["DiagnosisCodeEncoded"] = encoder.fit_transform(
    final_df["PrimaryDiagnosisDescription"]
)

# Drop unnecessary columns if needed
final_df = final_df.drop(
    columns=["PatientDateOfBirth", "AdmissionStartDate", "AdmissionEndDate"]
)

ValueError: time data '15:49.6' does not match format '%Y-%m-%d'